### Imports and paths

In [1]:
# Common imports
import os
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
import torch
import pandas as pd

# Repo imports
from util import losses

In [2]:
# Setting paths
root = '/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-clean/'

# Fixed paths
ckpt_dir = os.path.join(root, 'ckpts')
data_dir = os.path.join(root, 'data')

### Table1 (simulator)

In [7]:
# Reconstruction evaluation: Image-wise L1 and LPIPS, totem pose L1
def image_l1(imgA_path, imgB_path, y_valid=None):
    imgA = (cv2.imread(imgA_path)).astype(float)
    imgB = (cv2.imread(imgB_path)).astype(float)
    if y_valid:
        l1 = np.mean(np.abs(imgA-imgB)[:y_valid])/255
    else:
        l1 = np.mean(np.abs(imgA-imgB))/255
    return l1

def image_LPIPS(imgA_path, imgB_path, lpips_loss, y_valid=None):
    imgA = Image.open(imgA_path)
    imgB = Image.open(imgB_path)
    if y_valid:
        w, h = imgA.size
        imgA_tensor = transform(imgA.crop((0, 0, w, y_valid)))
        imgB_tensor = transform(imgB.crop((0, 0, w, y_valid)))
    else:
        imgA_tensor = transform(imgA)
        imgB_tensor = transform(imgB)
#     if y_valid:
#         w, h = imgA.size
#         imgA = imgA.crop((0, 0, w, y_valid))
#         imgB = imgB.crop((0, 0, w, y_valid))
#     imgA_tensor = transform(imgA)
#     imgB_tensor = transform(imgB)
    with torch.no_grad():
        out_lpips = lpips_loss(imgA_tensor[None], imgB_tensor[None])
    lpips_dist = np.mean(out_lpips[0][0].numpy())
    return lpips_dist

def load_totem_pose(ckpt_name):
    fnames = os.listdir(os.path.join(ckpt_dir, ckpt_name))
    fnames = sorted([fname for fname in fnames if 'tar' in fname])
    ckpt = torch.load(os.path.join(ckpt_dir, ckpt_name, fnames[-1]))
    totem_pose = ckpt['totem_pos'].detach().cpu().numpy()
    return totem_pose * 100 # totem_pose was scaled down by 100 for training, now scaling back

def totem_pose_l1(est, gt):
    return np.mean(np.abs(est-gt))

In [4]:
# Set up loss and transform for LPIPS
os.environ['HOME'] = root
lpips_loss = losses.LPIPS_Loss()
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: /data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/nerf-pytorch/venv/lib/python3.6/site-packages/lpips/weights/v0.1/vgg.pth


In [6]:
# Evaluation
ckpt_names = ['mitsuba_initial_pose', 'mitsuba_joint_pose', 'mitsuba_gt_pose', 
             'envmap_initial_pose', 'envmap_joint_pose', 'envmap_gt_pose']

l1_dists = []
lpips_dists = []
totem_l1_dists = []
for ckpt_name in ckpt_names:
    # Calculate L1 and LPIPS
    recon_path = os.path.join(ckpt_dir, ckpt_name, 'recon.png')
    gt_path = os.path.join(ckpt_dir, ckpt_name, 'gt.jpg')
    l1_dists.append(round(image_l1(recon_path, gt_path), 3))
    lpips_dists.append(round(image_LPIPS(recon_path, gt_path, lpips_loss), 3))    
    # Calculate totem L1
    gt_totem_pose = np.load(os.path.join(data_dir, ckpt_name.split('_')[0], 'gt_totem_pose.npy'))
    if 'joint' in ckpt_name:
        est_totem_pose = load_totem_pose(ckpt_name)
    elif 'initial' in ckpt_name:
        est_totem_pose = np.load(os.path.join(data_dir, ckpt_name.split('_')[0], 'initial_totem_pose.npy'))
    else:
        est_totem_pose = gt_totem_pose
        
    totem_l1_dists.append(round(totem_pose_l1(est_totem_pose, gt_totem_pose), 3))
        
    

In [7]:
# Print table
table1_sim = pd.DataFrame(np.transpose([l1_dists, lpips_dists, totem_l1_dists]),
                          index=ckpt_names,
                          columns=["L1", "LPIPS", "totem L1"])
table1_sim

,L1,LPIPS,totem L1
mitsuba_initial_pose,0.057,0.658,0.008
mitsuba_joint_pose,0.054,0.645,0.108
mitsuba_gt_pose,0.047,0.625,0.000
envmap_initial_pose,0.173,0.617,0.060
envmap_joint_pose,0.103,0.520,0.027
envmap_gt_pose,0.040,0.476,0.000


### Table1 (real images, excluding totem area)

In [13]:
# Getting valid y range
def get_y_valid(mask_path):
    mask = cv2.imread(mask_path)[..., 0]
    ys, xs = np.where(mask > 0)
    return min(ys)

In [15]:
ckpt_names = [ckpt_name for ckpt_name in os.listdir(ckpt_dir) if ('mitsuba' not in ckpt_name) and ('envmap' not in ckpt_name)]
ckpt_names = sorted(ckpt_names)

initial_l1_dists = []
initial_lpips_dists = []
joint_l1_dists = []
joint_lpips_dists = []
for ckpt_name in ckpt_names:
    print(ckpt_name)
    recon_path = os.path.join(ckpt_dir, ckpt_name, 'eval', 'recon.png')
    gt_path = os.path.join(ckpt_dir, ckpt_name, 'eval', 'gt.png')
    totem_mask_path = os.path.join(ckpt_dir, ckpt_name, 'eval', 'totem_mask.png')
    y_valid = get_y_valid(totem_mask_path)
    y_valid = y_valid // 3 * 3 # somehow for lpips, height must be multiples of 3
    l1_dist = round(image_l1(recon_path, gt_path, y_valid), 3)
    lpips_dist = round(image_LPIPS(recon_path, gt_path, lpips_loss, y_valid), 3)
    if 'initial' in ckpt_name:
        initial_l1_dists.append(l1_dist)
        initial_lpips_dists.append(lpips_dist)
    elif 'joint' in ckpt_name:
        joint_l1_dists.append(l1_dist)
        joint_lpips_dists.append(lpips_dist)


JT8A8283c_initial_pose
JT8A8283c_joint_pose
JT8A8287c_initial_pose
JT8A8287c_joint_pose
JT8A8291c_initial_pose
JT8A8291c_joint_pose
JT8A8292c_initial_pose
JT8A8292c_joint_pose
JT8A8293c_initial_pose
JT8A8293c_joint_pose
JT8A8297c_initial_pose
JT8A8297c_joint_pose
JT8A8299c_initial_pose
JT8A8299c_joint_pose
JT8A8351c_initial_pose
JT8A8351c_joint_pose
JT8A8352c_initial_pose
JT8A8352c_joint_pose
JT8A8357c_initial_pose
JT8A8357c_joint_pose
JT8A8358c_initial_pose
JT8A8358c_joint_pose
JT8A8359c_initial_pose
JT8A8359c_joint_pose
JT8A8360c_initial_pose
JT8A8360c_joint_pose
JT8A8362c_initial_pose
JT8A8362c_joint_pose
JT8A8365c_initial_pose
JT8A8365c_joint_pose
JT8A8366c_initial_pose
JT8A8366c_joint_pose
JT8A8367c_initial_pose
JT8A8367c_joint_pose
JT8A8368c_initial_pose
JT8A8368c_joint_pose
JT8A8369c_initial_pose
JT8A8369c_joint_pose
JT8A8370c_initial_pose
JT8A8370c_joint_pose
JT8A8371c_initial_pose
JT8A8371c_joint_pose
JT8A8372c_initial_pose
JT8A8372c_joint_pose
JT8A8373c_initial_pose
JT8A8373c

TypeError: 'NoneType' object is not subscriptable

In [20]:
# Print table
table1_real = pd.DataFrame(np.round([[np.mean(initial_l1_dists), np.mean(initial_lpips_dists)],
                                    [np.mean(joint_l1_dists), np.mean(joint_lpips_dists)]], 3),
                          index=["real_initial_pose", "real_joint_pose"],
                          columns=["L1", "LPIPS"])
table1_real

,L1,LPIPS
real_initial_pose,0.149,0.644
real_joint_pose,0.109,0.586


### Table2